A multiple choice task is similar to question answering, except several candidate answers are provided along with a context and the model is trained to select the correct answer.

This guide shows how to:

1. Finetune BERT on the regular configuration of the SWAG dataset to select the best answer given multiple options and some context.
2. Use finetuned model for inference.

# Libraries

In [ ]:
pip install transformers datasets evaluate

In [ ]:
from datasets import load_dataset

# Load Data

In [ ]:
swag = load_dataset("swag", "regular")

In [ ]:
# Inspection
# sent1 and sent2: these fields show how a sentence starts (put the two together, you get the startphrase field).
# ending: suggests a possible ending for how a sentence can end, but only one of them is correct.
# label: identifies the correct sentence ending.
swag["train"][0]